In [ ]:
import json
import sqlite3
from datetime import datetime

from data_api import youtube
from helpers import dict_search

In [ ]:
with open("./key.json") as f:
        DEVELOPER_KEY = json.load(f)['key']
    
yt = youtube(DEVELOPER_KEY)

In [ ]:
dt = datetime.now()

raw_categories = yt.VideoCategories('GB')
categories = dict_search(raw_categories, ["id", "title", "assignable"])
    
for cat in categories:
    cat['region'] = 'GB'
    cat['time_updated'] = dt

with sqlite3.connect('yt_sentiment.db') as conn:
    sql = "INSERT OR REPLACE INTO \
        categories(category_id, title, assignable, region, time_updated) \
        VALUES(:id, :title, :assignable, :region, :time_updated)"
    
    conn.executemany(sql, categories)